In [1]:
from PIL import Image
import os, glob, sys, numpy as np
from sklearn.model_selection import train_test_split
from Save_dataset_to_npy import categories
import matplotlib.pyplot as plt

import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend.tensorflow_backend as K
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import cv2

tf.__version__

[]


ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [2]:
x_train, x_test, y_train, y_test = np.load('./document_image_data.npy', allow_pickle = True)

print(x_train.shape)
print(x_train.shape[0])
print(x_test.shape)
print(x_test.shape[0])
print(y_train.shape)
print(y_train.shape[0])
print(y_test.shape)
print(y_test.shape[0])
x_train.shape[1:]

(314, 224, 224, 3)
314
(105, 224, 224, 3)
105
(314, 6)
314
(105, 6)
105


(224, 224, 3)

In [3]:
nb_classes = len(categories)
print(categories)
batch_size = 16
epochs = 25

['dog', 'eagle', 'elephant', 'horse', 'monkey', 'tiger']


In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [5]:
datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
                             samplewise_center=False,  # set each sample mean to 0
                             featurewise_std_normalization=False,  # divide inputs by std of the dataset
                             samplewise_std_normalization=False,  # divide each input by its std
                             zca_whitening=False,  # apply ZCA whitening
                             zca_epsilon=1e-06,  # epsilon for ZCA whitening
                             rotation_range=1,  # randomly rotate images in the range (degrees, 0 to 180)
                             # randomly shift images horizontally (fraction of total width)
                             width_shift_range=0.05,
                             # randomly shift images vertically (fraction of total height)
                             height_shift_range=0.05,
                             shear_range=0.05,  # set range for random shear
                             zoom_range=0.05,  # set range for random zoom
                             channel_shift_range=0.,  # set range for random channel shifts
                             # set mode for filling points outside the input boundaries
                             fill_mode='nearest',
                             cval=0.,  # value used for fill_mode = "constant"
                             horizontal_flip=False,  # randomly flip images y축으로 반사
                             vertical_flip=False,  # randomly flip images x축으로 반사
                             # set rescaling factor (applied before any other transformation)
                             rescale=1./255,
                             # set function that will be applied on each input
                             preprocessing_function=None,
                             # image data format, either "channels_first" or "channels_last"
                             data_format=None,
                             # fraction of images reserved for validation (strictly between 0 and 1)
                             validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [6]:
filepath = "./modelSave/saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [15]:
# for example,
n_kernels = 32
model = Sequential()

# b, 32, 32, 3
model.add(Conv2D(n_kernels, (3, 3), activation='relu', kernel_initializer='he_normal', 
                 padding='valid', input_shape=x_train.shape[1:]))
model.add(Conv2D(n_kernels*2, (3, 3), activation='relu', kernel_initializer='he_normal', 
                 padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# b, 16, 16, 64
model.add(Conv2D(n_kernels*2, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(Conv2D(n_kernels*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(n_kernels*4, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(Conv2D(n_kernels*8, (3, 3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# b, 8, 8, 128
model.add(Conv2D(n_kernels*8, (3,3), activation='relu', kernel_initializer='he_normal', padding='valid'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

# [중요] GlobalAveragePooling2D() 플레툰 할 때 쫙 펼쳐서하는 것이 아니라 앞에 단계의 Conv2D에서 도출될 피쳐맵(여기선 3*3*256)에서 각 피쳐(3*3)의 평균으로 256노드에 순차적으로 넣어줌)
# 그런데 플래툰하면 256노드가 아니라 3*3*256 노드가 생김
model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation='softmax', kernel_initializer='glorot_uniform'))

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 220, 220, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 106, 106, 128)     73856     
_________________________________________________________________
batch_normalization_5 (Batch (None, 106, 106, 128)    

In [20]:
opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])

In [22]:
# Let's train the model using RMSprop
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                    validation_data=(x_test, y_test),
                    workers=4, callbacks=[checkpoint])

Epoch 1/25
20/20 [==============================] - 5s 235ms/step - loss: 0.0379 - acc: 0.9936 - val_loss: 13.5085 - val_acc: 0.1619

Epoch 00001: saving model to ./modelSave/saved-model-01-0.16.hdf5
Epoch 2/25
20/20 [==============================] - 4s 190ms/step - loss: 0.0477 - acc: 0.9873 - val_loss: 13.8155 - val_acc: 0.1429

Epoch 00002: saving model to ./modelSave/saved-model-02-0.14.hdf5
Epoch 3/25
20/20 [==============================] - 4s 191ms/step - loss: 0.0167 - acc: 1.0000 - val_loss: 11.7393 - val_acc: 0.2667

Epoch 00003: saving model to ./modelSave/saved-model-03-0.27.hdf5
Epoch 4/25
20/20 [==============================] - 4s 190ms/step - loss: 0.0093 - acc: 1.0000 - val_loss: 13.8155 - val_acc: 0.1429

Epoch 00004: saving model to ./modelSave/saved-model-04-0.14.hdf5
Epoch 5/25
20/20 [==============================] - 4s 190ms/step - loss: 0.0401 - acc: 0.9936 - val_loss: 12.9235 - val_acc: 0.1810

Epoch 00005: saving model to ./modelSave/saved-model-05-0.18.hdf5


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\n 테스트 정확도:', test_acc)

In [ ]:
pred = model.predict(x_test)

In [ ]:
print(pred[0])
print(y_test[0])
print(np.argmax(pred[0]))

In [ ]:
def plot_image(i, pred, y_test, x_test):
    pred, y_test, x_test = pred[i], y_test[i], x_test[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    plt.imshow(x_test, cmap=plt.cm.binary)
    
    pred_label = np.argmax(pred)
    test_label = np.argmax(y_test)
    print(pred_label)
    print(test_label)
    if pred_label == test_label:
        color = 'blue'
    else:
        color = 'red'
    plt.xlabel("{} {:2.0f}% ({})".format(categories[pred_label], 100*np.max(pred), categories[test_label]),color=color)

In [ ]:
def plot_image(i, pred, y_test, x_test):
    pred, y_test, x_test = pred[i], y_test[i], x_test[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    plt.imshow(x_test, cmap=plt.cm.binary)
    
    pred_label = np.argmax(pred)
    test_label = np.argmax(y_test)
    print(pred_label)
    print(test_label)
    if pred_label == test_label:
        color = 'blue'
    else:
        color = 'red'
    plt.xlabel("{} {:2.0f}% ({})".format(categories[pred_label], 100*np.max(pred), categories[test_label]),color=color)

In [ ]:
i = 4
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, pred, y_train, x_train)

In [ ]:
i = 6
plt.xticks([])
plt.yticks([])
plt.imshow(x_train[i], cmap=plt.cm.binary)
a = int(np.argmax(y_train[i]))
print(categories[a])